# get subset of data

In [2]:
import os
import random
from pathlib import Path

dir = Path('dataset/cifar10-small')

for subset in ['train', 'test']:
    for class_ in os.listdir(dir / subset):
        path= dir / subset / class_
        
        image_files = os.listdir(path)
        print(subset, '-', class_, '-', len(image_files))

train - deer - 1000
train - cat - 1000
train - horse - 1000
train - dog - 1000
train - automobile - 1000
train - bird - 1000
train - frog - 1000
train - truck - 1000
train - airplane - 1000
train - ship - 1000
test - deer - 200
test - cat - 200
test - horse - 200
test - dog - 200
test - automobile - 200
test - bird - 200
test - frog - 200
test - truck - 200
test - airplane - 200
test - ship - 200


In [3]:
dataset_size= 0.2
DELETE= False

if DELETE:
    for subset in ['train', 'test']:
        for class_ in os.listdir(dir / subset):
            path= dir / subset / class_
            
            image_files = os.listdir(path)

            # get random images based on intended dataset_size
            num_images_to_delete = int((1-dataset_size) * len(image_files))
            images_to_delete = random.sample(image_files, num_images_to_delete)

            # delete images
            for image in images_to_delete:
                image_path = os.path.join(path, image)
                os.remove(image_path)

In [4]:
dir = Path('dataset/cifar10-small')

for subset in ['train', 'test']:
    for class_ in os.listdir(dir / subset):
        path= dir / subset / class_
        
        image_files = os.listdir(path)
        print(subset, '-', class_, '-', len(image_files))

train - deer - 1000
train - cat - 1000
train - horse - 1000
train - dog - 1000
train - automobile - 1000
train - bird - 1000
train - frog - 1000
train - truck - 1000
train - airplane - 1000
train - ship - 1000
test - deer - 200
test - cat - 200
test - horse - 200
test - dog - 200
test - automobile - 200
test - bird - 200
test - frog - 200
test - truck - 200
test - airplane - 200
test - ship - 200


# get dataset as array

In [5]:
import os
import numpy as np
from PIL import Image

In [6]:
def load_image_dataset(root_dir, shuffle= True):
    
    class_labels = []
    images = []
    
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if os.path.isdir(class_path):
            class_label = class_dir
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                image = Image.open(image_path)
                image = np.array(image)
                class_labels.append(class_label)
                images.append(image)

    images = np.array(images)
    class_labels = np.array(class_labels)
    
    if shuffle:
        shuffle_indices = np.random.permutation(len(images))
        images = images[shuffle_indices]
        class_labels = class_labels[shuffle_indices]
    
    return images, class_labels

train = 'dataset/cifar10-small/train'
x_train, y_train= load_image_dataset(train)

test = 'dataset/cifar10-small/test'
x_test, y_test= load_image_dataset(test)

# get validation set
perc_vals= 0.2
n_vals= int(len(x_train) * perc_vals)

x_val= x_train[:n_vals]
y_val= y_train[:n_vals]

x_train= x_train[n_vals:]
y_train= y_train[n_vals:]

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(8000, 32, 32, 3) (8000,)
(2000, 32, 32, 3) (2000,)
(2000, 32, 32, 3) (2000,)


# with TF

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Activation, Flatten, Dropout

2023-07-16 08:17:43.926414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# normalize the data
mean= np.mean(x_train)
std= np.std(x_train)

x_train_clean= (x_train - mean)/std
x_val_clean= (x_val - mean)/std
x_test_clean= (x_test - mean)/std

# convert labels to one-hot
encoder= OneHotEncoder(sparse_output= False, drop= None, handle_unknown= 'ignore')
encoder.fit(y_train.reshape(-1, 1))
y_train_clean= encoder.transform(y_train.reshape(-1, 1))
y_val_clean= encoder.transform(y_val.reshape(-1, 1))
y_test_clean= encoder.transform(y_test.reshape(-1, 1))

In [9]:
datagen= ImageDataGenerator(
    rotation_range= 15, 
    width_shift_range= 0.1, 
    height_shift_range= 0.1,
    horizontal_flip= True,
    vertical_flip= True
)
datagen.fit(x_train_clean)
datagen

In [10]:
x_train_clean.shape[1:]

(32, 32, 3)

In [11]:
y_train_clean.shape[-1]

10

In [12]:
base_hidden_unit= 32
num_classes= y_train_clean.shape[-1]
weight_decay= 1e-4
model= keras.Sequential()

# conv 1
model.add(Conv2D(filters= base_hidden_unit, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 input_shape= x_train_clean.shape[1:]
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 2
model.add(Conv2D(filters= base_hidden_unit, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.2))

# conv 3
model.add(Conv2D(filters= base_hidden_unit * 2, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 4
model.add(Conv2D(filters= base_hidden_unit *2, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.3))

# conv 5
model.add(Conv2D(filters= base_hidden_unit * 4, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# conv 6
model.add(Conv2D(filters= base_hidden_unit * 4, 
                 kernel_size= (3, 3),
                 padding= 'same', 
                 kernel_regularizer= keras.regularizers.l2(weight_decay),
                 ))
model.add(Activation('relu'))
model.add(BatchNormalization())

# pool + dropout
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Dropout(0.4))

# fc
model.add(Flatten())
model.add(Dense(num_classes, activation= 'softmax'))

model.summary()

2023-07-16 08:17:46.530468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 08:17:46.537650: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

In [13]:
TRAIN= False
batch_size= 16
epochs= 100
model_path= 'output/model.hdf5'

checkpoint= ModelCheckpoint(filepath= model_path, save_best_only= True, verbose= 1)
optimizer= keras.optimizers.Adam(lr=0.0001,
                                 weight_decay= 1e-6)

model.compile(loss='categorical_crossentropy', 
              optimizer= optimizer,
              metrics= ['accuracy'])

if TRAIN:
    history= model.fit_generator(
        datagen.flow(x_train_clean, y_train_clean, batch_size= batch_size),
        callbacks= checkpoint, 
        steps_per_epoch= x_train_clean.shape[0] // batch_size, 
        epochs= epochs,
        verbose= 1, 
        validation_data= (x_val_clean, y_val_clean)
        )

In [16]:
saved_model= keras.models.load_model(model_path)
saved_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                      

In [17]:
scores = saved_model.evaluate(x_test_clean, 
                              y_test_clean, 
                              batch_size=128, 
                              verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

16/16 [==============================] - 5s 252ms/step - loss: 1.0534 - accuracy: 0.6870

Test result: 68.700 loss: 1.053


# with pytorch

In [20]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [21]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(8000, 32, 32, 3) (8000,)
(2000, 32, 32, 3) (2000,)
(2000, 32, 32, 3) (2000,)
